In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\| **Autograd** \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Automatic Differentiation with `torch.autograd`
===============================================

When training neural networks, the most frequently used algorithm is
**back propagation**. In this algorithm, parameters (model weights) are
adjusted according to the **gradient** of the loss function with respect
to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation
engine called `torch.autograd`. It supports automatic computation of
gradient for any computational graph.

Consider the simplest one-layer neural network, with input `x`,
parameters `w` and `b`, and some loss function. It can be defined in
PyTorch in the following manner:

`torch.autograd`를 사용한 자동 미분
===============================================

신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 **역전파**입니다. 이 알고리즘에서는 주어진 매개변수에 대한 손실 함수의 **기울기**에 따라 매개변수(모델 가중치)가 조정됩니다.

이러한 기울기를 계산하기 위해 PyTorch는 `torch.autograd`라는 내장 미분 엔진을 가지고 있습니다. 이는 모든 계산 그래프에 대한 기울기의 자동 계산을 지원합니다.

입력 `x`, 매개변수 `w`와 `b`, 그리고 일부 손실 함수를 가진 가장 간단한 단층 신경망을 고려해봅시다. 이는 PyTorch에서 다음과 같은 방식으로 정의될 수 있습니다:

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

Tensors, Functions and Computational graph
==========================================

This code defines the following **computational graph**:

![](https://pytorch.org/tutorials/_static/img/basics/comp-graph.png)

In this network, `w` and `b` are **parameters**, which we need to
optimize. Thus, we need to be able to compute the gradients of loss
function with respect to those variables. In order to do that, we set
the `requires_grad` property of those tensors.

텐서, 함수 및 계산 그래프
==========================================

이 코드는 다음과 같은 **계산 그래프**를 정의합니다:

![](https://pytorch.org/tutorials/_static/img/basics/comp-graph.png)

이 네트워크에서 `w`와 `b`는 우리가 최적화해야 하는 **매개변수**입니다. 따라서 우리는 이 변수들에 대한 손실 함수의 기울기를 계산할 수 있어야 합니다. 이를 위해 해당 텐서들의 `requires_grad` 속성을 설정합니다.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>You can set the value of <code>requires_grad</code> when creating atensor, or later by using <code>x.requires_grad_(True)</code> method.</p>
</div>

---

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>참고:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>텐서를 생성할 때 <code>requires_grad</code> 값을 설정하거나, 나중에 <code>x.requires_grad_(True)</code> 메서드를 사용하여 설정할 수 있습니다.</p>
</div>

A function that we apply to tensors to construct computational graph is
in fact an object of class `Function`. This object knows how to compute
the function in the *forward* direction, and also how to compute its
derivative during the *backward propagation* step. A reference to the
backward propagation function is stored in `grad_fn` property of a
tensor. You can find more information of `Function` [in the
documentation](https://pytorch.org/docs/stable/autograd.html#function).

---

우리가 텐서에 적용하여 계산 그래프를 구성하는 함수는 사실 `Function` 클래스의 객체입니다. 이 객체는 *순방향*으로 함수를 계산하는 방법과 *역전파* 단계에서 함수의 미분을 계산하는 방법을 알고 있습니다. 역전파 함수에 대한 참조는 텐서의 `grad_fn` 속성에 저장됩니다. `Function`에 대한 자세한 정보는 [문서](https://pytorch.org/docs/stable/autograd.html#function)에서 찾을 수 있습니다.

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f7ad10de5e0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f7ad10de970>


Computing Gradients
===================

To optimize weights of parameters in the neural network, we need to
compute the derivatives of our loss function with respect to parameters,
namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of `x` and
`y`. To compute those derivatives, we call `loss.backward()`, and then
retrieve the values from `w.grad` and `b.grad`:

---

경사도 계산
===================

신경망의 매개변수 가중치를 최적화하려면 손실 함수의 매개변수에 대한 도함수를 계산해야 합니다. 즉, 고정된 `x`와 `y` 값에 대해 $\frac{\partial loss}{\partial w}$ 및 $\frac{\partial loss}{\partial b}$를 계산해야 합니다. 이러한 도함수를 계산하려면 `loss.backward()`를 호출한 다음 `w.grad` 및 `b.grad`에서 값을 가져옵니다:


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1991, 0.0042, 0.3260],
        [0.1991, 0.0042, 0.3260],
        [0.1991, 0.0042, 0.3260],
        [0.1991, 0.0042, 0.3260],
        [0.1991, 0.0042, 0.3260]])
tensor([0.1991, 0.0042, 0.3260])


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<ul>
<li>We can only obtain the <code>grad</code> properties for the leafnodes of the computational graph, which have <code>requires_grad</code> propertyset to <code>True</code>. For all other nodes in our graph, gradients will not beavailable.- We can only perform gradient calculations using<code>backward</code> once on a given graph, for performance reasons. If we needto do several <code>backward</code> calls on the same graph, we need to pass<code>retain_graph=True</code> to the <code>backward</code> call.</li>
</ul>
</div>

---

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>참고:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<ul>
<li>계산 그래프의 리프 노드에 대해서만 <code>grad</code> 속성을 얻을 수 있으며, 이러한 노드는 <code>requires_grad</code> 속성이 <code>True</code>로 설정되어 있습니다. 그래프의 다른 모든 노드에 대해서는 경사도를 얻을 수 없습니다.</li>
<li>성능상의 이유로 주어진 그래프에서 <code>backward</code>를 한 번만 사용할 수 있습니다. 동일한 그래프에서 여러 번 <code>backward</code> 호출을 해야 하는 경우 <code>retain_graph=True</code>를 <code>backward</code> 호출에 전달해야 합니다.</li>
</ul>
</div>


Disabling Gradient Tracking
===========================

By default, all tensors with `requires_grad=True` are tracking their
computational history and support gradient computation. However, there
are some cases when we do not need to do that, for example, when we have
trained the model and just want to apply it to some input data, i.e. we
only want to do *forward* computations through the network. We can stop
tracking computations by surrounding our computation code with
`torch.no_grad()` block:

---

경사도 추적 비활성화
===========================

기본적으로 `requires_grad=True`가 설정된 모든 텐서는 계산 기록을 추적하고 경사도 계산을 지원합니다. 하지만 모델을 훈련한 후 단순히 일부 입력 데이터에 적용하여 *순방향* 계산만 수행하고 싶을 때처럼 경사도 추적이 필요 없는 경우가 있습니다. 이러한 경우 `torch.no_grad()` 블록으로 계산 코드를 감싸서 계산 추적을 중지할 수 있습니다:

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the `detach()` method
on the tensor:

---

같은 결과를 얻는 또 다른 방법은 텐서에서 `detach()` 메서드를 사용하는 것입니다.


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:

:   -   To mark some parameters in your neural network as **frozen
        parameters**.
    -   To **speed up computations** when you are only doing forward
        pass, because computations on tensors that do not track
        gradients would be more efficient.

---

기울기 추적을 비활성화해야 할 이유는 다음과 같습니다:

- 신경망의 일부 매개변수를 **고정 매개변수** (frozen parameters)로 표시하기 위해.
- 오직 순전파 (forward pass)만 수행할 때 **계산 속도를 높이기 위해**, 기울기를 추적하지 않는 텐서에 대한 계산이 더 효율적이기 때문입니다.

More on Computational Graphs
============================

Conceptually, autograd keeps a record of data (tensors) and all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

-   run the requested operation to compute a resulting tensor
-   maintain the operation's *gradient function* in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG
root. `autograd` then:

-   computes the gradients from each `.grad_fn`,
-   accumulates them in the respective tensor's `.grad` attribute
-   using the chain rule, propagates all the way to the leaf tensors.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>An important thing to note is that the graph is recreated from scratch; after each<code>.backward()</code> call, autograd starts populating a new graph. This isexactly what allows you to use control flow statements in your model;you can change the shape, size and operations at every iteration ifneeded.</p>
</div>

---

**계산 그래프에 대한 추가 설명**  
============================  

개념적으로, autograd는 데이터(텐서)와 실행된 모든 연산(결과적으로 생성된 새로운 텐서와 함께)의 기록을 방향성 비순환 그래프(DAG)로 유지합니다. 이 DAG에서 리프(leaf)는 입력 텐서이고, 루트(root)는 출력 텐서입니다. 이 그래프를 루트에서 리프로 추적함으로써, 체인 룰을 사용하여 자동으로 기울기를 계산할 수 있습니다.

순전파(forward pass)에서는 autograd가 두 가지 작업을 동시에 수행합니다:

- 요청된 연산을 실행하여 결과 텐서를 계산합니다.
- DAG에서 연산의 *기울기 함수(gradient function)*를 유지합니다.

역전파(backward pass)는 DAG 루트에서 `.backward()`가 호출될 때 시작됩니다. 그때 `autograd`는:

- 각 `.grad_fn`에서 기울기를 계산합니다.
- 해당 텐서의 `.grad` 속성에 기울기를 누적합니다.
- 체인 룰을 사용하여 리프 텐서까지 전파합니다.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>참고:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">  
<p>중요한 점은 그래프가 매번 처음부터 새로 생성된다는 것입니다. 각 <code>.backward()</code> 호출 후, autograd는 새로운 그래프를 채우기 시작합니다. 이것은 모델에서 제어 흐름 문을 사용할 수 있게 해주는 중요한 요소이며, 필요에 따라 매 반복마다 형태, 크기 및 연산을 변경할 수 있습니다.</p>
</div>

Optional Reading: Tensor Gradients and Jacobian Products
========================================================

In many cases, we have a scalar loss function, and we need to compute
the gradient with respect to some parameters. However, there are cases
when the output function is an arbitrary tensor. In this case, PyTorch
allows you to compute so-called **Jacobian product**, and not the actual
gradient.

For a vector function $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ and
$\vec{y}=\langle y_1,\dots,y_m\rangle$, a gradient of $\vec{y}$ with
respect to $\vec{x}$ is given by **Jacobian matrix**:

$$\begin{aligned}
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
\end{aligned}$$

Instead of computing the Jacobian matrix itself, PyTorch allows you to
compute **Jacobian Product** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$. This is achieved by calling `backward` with $v$ as
an argument. The size of $v$ should be the same as the size of the
original tensor, with respect to which we want to compute the product:

---

**선택적 읽기: 텐서 기울기와 야코비안 곱**  
========================================================  

많은 경우, 우리는 스칼라 손실 함수가 있으며, 특정 매개변수에 대한 기울기를 계산해야 합니다. 그러나 출력 함수가 임의의 텐서인 경우도 있습니다. 이 경우, PyTorch는 실제 기울기 대신 **자코비안 곱**(Jacobian product)을 계산할 수 있도록 합니다.

벡터 함수 $\vec{y}=f(\vec{x})$에 대해, 여기서 $\vec{x}=\langle x_1,\dots,x_n\rangle$이고 $\vec{y}=\langle y_1,\dots,y_m\rangle$일 때, $\vec{y}$의 $\vec{x}$에 대한 기울기는 **자코비안 행렬**(Jacobian matrix)로 주어집니다:

$$\begin{aligned}
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
\end{aligned}$$

자코비안 행렬 자체를 계산하는 대신, PyTorch는 주어진 입력 벡터 $v=(v_1, \dots, v_m)$에 대해 **자코비안 곱** $v^T\cdot J$를 계산할 수 있도록 합니다. 이는 $v$를 인수로 하여 `backward`를 호출함으로써 수행됩니다. $v$의 크기는 곱을 계산하고자 하는 원래 텐서의 크기와 같아야 합니다.


In [6]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


In [7]:
inp = torch.eye(4, 5, requires_grad=True) # 입력 텐서 생성(4*5의 단위행렬[대각선 원소가 1이고 나머지 0])
out = (inp + 1).pow(2).t() # 출력 텐서 계산 
out.backward(torch.ones_like(out), retain_graph=True) # 첫 번째 역전파 호출
print(f"First call\n{inp.grad}") # 첫 번째 기울기 출력
out.backward(torch.ones_like(out), retain_graph=True) # 두 번째 역전파 호출
print(f"\nSecond call\n{inp.grad}") # 두 번째 기울기 출력
inp.grad.zero_() # 기울기 초기화
out.backward(torch.ones_like(out), retain_graph=True) # 세 번째 역전파 호출
print(f"\nCall after zeroing gradients\n{inp.grad}") # 기울기 초기화한 후의 세 번째 기울기 출력

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


Notice that when we call `backward` for the second time with the same
argument, the value of the gradient is different. This happens because
when doing `backward` propagation, PyTorch **accumulates the
gradients**, i.e. the value of computed gradients is added to the `grad`
property of all leaf nodes of computational graph. If you want to
compute the proper gradients, you need to zero out the `grad` property
before. In real-life training an *optimizer* helps us to do this.

---

두 번째로 같은 인수로 `backward`를 호출할 때 기울기 값이 다르다는 점에 유의해야 합니다. 이는 역전파를 수행할 때 PyTorch가 **기울기를 누적**(accumulates the gradients)하기 때문입니다. 즉, 계산된 기울기의 값이 계산 그래프의 모든 리프 노드의 `grad` 속성에 추가됩니다. 올바른 기울기를 계산하려면 먼저 `grad` 속성을 0으로 초기화해야 합니다. 실제 훈련에서는 **옵티마이저(optimizer)**가 이를 도와줍니다.

---

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>Previously we were calling <code>backward()</code> function withoutparameters. This is essentially equivalent to calling<code>backward(torch.tensor(1.0))</code>, which is a useful way to compute thegradients in case of a scalar-valued function, such as loss duringneural network training.</p>
</div>

---

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>참고:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>이전에는 <code>backward()</code> 함수를 매개변수 없이 호출했습니다. 이는 본질적으로 <code>backward(torch.tensor(1.0))</code>를 호출하는 것과 동일하며, 이는 신경망 훈련 중 손실과 같은 스칼라 값 함수의 기울기를 계산할 때 유용한 방법입니다.</p>
</div>

------------------------------------------------------------------------


Further Reading
===============

-   [Autograd
    Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)
